## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,2022-07-25 17:14:11,-24.7911,-50.0119,75.74,40,67,2.62,broken clouds
1,1,Rikitea,PF,2022-07-25 17:14:12,-23.1203,-134.9692,75.11,72,16,19.89,few clouds
2,2,Pacific Grove,US,2022-07-25 17:14:06,36.6177,-121.9166,60.03,91,100,3.00,overcast clouds
3,3,Tilichiki,RU,2022-07-25 17:14:12,60.4667,166.1000,47.34,85,60,3.49,broken clouds
4,4,Russell,US,2022-07-25 17:14:12,32.3502,-85.1999,91.96,70,75,0.00,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?89


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc metho# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,2022-07-25 17:14:11,-24.7911,-50.0119,75.74,40,67,2.62,broken clouds
1,1,Rikitea,PF,2022-07-25 17:14:12,-23.1203,-134.9692,75.11,72,16,19.89,few clouds
6,6,Kapaa,US,2022-07-25 17:14:13,22.0752,-159.3190,79.12,80,40,16.11,scattered clouds
8,8,Praia Da Vitoria,PT,2022-07-25 17:14:14,38.7333,-27.0667,79.75,86,39,3.11,light rain
9,9,Hasaki,JP,2022-07-25 17:14:14,35.7333,140.8333,79.29,93,88,15.50,overcast clouds
12,12,Vaini,TO,2022-07-25 17:14:15,-21.2000,-175.2000,71.76,88,40,5.75,scattered clouds
14,14,Bethel,US,2022-07-25 17:14:16,41.3712,-73.4140,86.49,69,100,10.36,heavy intensity rain
15,15,Sabang,ID,2022-07-25 17:14:16,5.8933,95.3214,79.72,80,98,12.50,overcast clouds
17,17,Vanimo,PG,2022-07-25 17:14:17,-2.6741,141.3028,77.14,91,60,5.39,broken clouds
18,18,Puerto Ayora,EC,2022-07-25 17:13:06,-0.7393,-90.3518,78.76,91,88,7.00,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                318
City                   318
Country                318
Date                   318
Lat                    318
Lng                    318
Max Temp               318
Humidity               318
Cloudiness             318
Wind Speed             318
Current Description    318
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,75.74,broken clouds,-24.7911,-50.0119,
1,Rikitea,PF,75.11,few clouds,-23.1203,-134.9692,
6,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,
8,Praia Da Vitoria,PT,79.75,light rain,38.7333,-27.0667,
9,Hasaki,JP,79.29,overcast clouds,35.7333,140.8333,
12,Vaini,TO,71.76,scattered clouds,-21.2000,-175.2000,
14,Bethel,US,86.49,heavy intensity rain,41.3712,-73.4140,
15,Sabang,ID,79.72,overcast clouds,5.8933,95.3214,
17,Vanimo,PG,77.14,broken clouds,-2.6741,141.3028,
18,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,


In [12]:
#  6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        #hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,75.74,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
1,Rikitea,PF,75.11,few clouds,-23.1203,-134.9692,People ThankYou
6,Kapaa,US,79.12,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Praia Da Vitoria,PT,79.75,light rain,38.7333,-27.0667,Hotel Branco I
9,Hasaki,JP,79.29,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
12,Vaini,TO,71.76,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Bethel,US,86.49,heavy intensity rain,41.3712,-73.4140,Courtyard by Marriott Danbury
15,Sabang,ID,79.72,overcast clouds,5.8933,95.3214,Hotel Citra
17,Vanimo,PG,77.14,broken clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
18,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))